In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from matplotalt import *

In [2]:
vistext_id_to_mpl_code = np.load("./vistext_id_to_mpl_code.npy", allow_pickle=True).item()
vistext_val_data = pd.read_json("./vistext_data_test.json")
#vistext_dev_data = pd.read_json("./vistext_data_validation.json")
#vistext_train_data = pd.read_json("./vistext_data_train.json")
vistext_id_to_captions = defaultdict(list)
vistext_id_to_chart_type = {}
def get_id_to_captions(row):
    vistext_id_to_captions[row["img_id"]].append({"L1": row["caption_L1"], "L2L3": row["caption_L2L3"]})
    vistext_id_to_chart_type[row["img_id"]] = row["L1_properties"][0]

vistext_val_data.apply(get_id_to_captions, axis=1)
#vistext_dev_data.apply(get_id_to_captions, axis=1)
#vistext_train_data.apply(get_id_to_captions, axis=1)
#np.save("./vistext_test_id_to_captions", vistext_id_to_captions)

0       None
1       None
2       None
3       None
4       None
        ... 
1265    None
1266    None
1267    None
1268    None
1269    None
Length: 1270, dtype: object

In [4]:
#print(len(vistext_id_to_captions))
n_passed = 0
n_errors = 0
#vistext_id_to_captions = np.load("./evaluation/vistext_to_mpl/vistext_id_to_captions.npy", allow_pickle=True).item()
vistext_id_to_matplotalt_captions = {}
for chart_id in tqdm(vistext_id_to_captions.keys()):
    #print(vistext_id_to_mpl_code.keys())
    try:
        plt.clf()
        mpl_code = vistext_id_to_mpl_code[str(chart_id)]
        chart_type = vistext_id_to_chart_type[chart_id]
        exec(mpl_code)
        #print(chart_id)
        #print(mpl_code)
        #plt.show()
        #plt.gcf().savefig(f"./matplotlib_ver_imgs/{chart_id}.png", bbox_inches='tight')
        # Only include min and max stats as others (like linear fit, std rarely or do not occur in vistext captions)
        # Also exclude color descriptions since they are different in the matplotlib version of vistext figures
        matplotalt_caption = show_with_alt(desc_level=3, methods=[], stats=["min", "max"], max_color_desc_count=0, return_alt=True)
        vistext_id_to_matplotalt_captions[chart_id] = matplotalt_caption
        n_passed += 1
    except Exception as e:
        #raise e
        print(chart_id)
        print(mpl_code)
        n_errors += 1
        print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")
np.save("./vistext_test_id_to_matplotalt_captions", vistext_id_to_matplotalt_captions)

100%|██████████| 882/882 [00:41<00:00, 21.34it/s]

Num passed: 882
Num errors: 0


<Figure size 640x480 with 0 Axes>

In [4]:
vistext_id_to_matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4_captions = np.load("./vistext_test_id_to_L3_gpt4_captions.npy", allow_pickle=True).item()

OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
model = "TURBO"
n_errors = 0
n_passed = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    if chart_id not in vistext_id_to_gpt4_captions:
        print(chart_id)
        try:
            mpl_code = vistext_id_to_mpl_code[str(chart_id)]
            chart_type = vistext_id_to_chart_type[chart_id]
            exec(mpl_code)
            # gpt4-turbo
            gpt4_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
                                            desc_level=3, stats=["min", "max"], max_color_desc_count=0,
                                            use_starter_alt_in_prompt=False, use_azure=True,
                                            include_colors=False)
            plt.clf()
            vistext_id_to_gpt4_captions[chart_id] = gpt4_caption
            np.save("./vistext_test_id_to_L3_gpt4_captions", vistext_id_to_gpt4_captions)
            n_passed += 1
        except Exception as e:
            #raise e
            print(chart_id)#, mpl_code)
            n_errors += 1
            plt.clf()
            print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")

  0%|          | 0/882 [00:00<?, ?it/s]

7360


 85%|████████▌ | 754/882 [00:07<00:01, 105.88it/s]

7361


100%|██████████| 882/882 [00:13<00:00, 67.20it/s] 

Num passed: 2
Num errors: 0


<Figure size 640x480 with 0 Axes>

In [5]:
vistext_id_to_matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4_alt_captions = {} #np.load("./vistext_test_id_to_L3_gpt4_alt_captions.npy", allow_pickle=True).item()

OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
model = "TURBO"
n_errors = 0
n_passed = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    if chart_id not in vistext_id_to_gpt4_alt_captions:
        #print(chart_id)
        try:
            mpl_code = vistext_id_to_mpl_code[str(chart_id)]
            chart_type = vistext_id_to_chart_type[chart_id]
            exec(mpl_code)
            # gpt4-turbo
            gpt4_alt_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
                                        desc_level=3, stats=["min", "max"], max_color_desc_count=0,
                                        use_starter_alt_in_prompt=True, use_azure=True,
                                        include_colors=False)
            plt.clf()
            vistext_id_to_gpt4_alt_captions[chart_id] = gpt4_alt_caption
            np.save("./vistext_test_id_to_L3_gpt4_alt_captions", vistext_id_to_gpt4_alt_captions)
            n_passed += 1
        except Exception as e:
            #raise e
            print(chart_id)#, mpl_code)
            n_errors += 1
            plt.clf()
            print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")

 56%|█████▌    | 490/882 [1:06:46<1:06:09, 10.13s/it]C:\Users\Kai\Desktop\matplotalt\matplotalt\matplotalt.py:538: UserWarning: Glyph 144 (\x90) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
 83%|████████▎ | 731/882 [1:36:10<14:32,  5.78s/it]  

7117
Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'filtered': True, 'detected': True}}}, 'code': 'content_filter', 'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: \r\nhttps://go.microsoft.com/fwlink/?linkid=2198766.", 'param': 'prompt', 'type': None}}


100%|██████████| 882/882 [1:55:13<00:00,  7.84s/it]

Num passed: 881
Num errors: 1


<Figure size 640x480 with 0 Axes>

In [6]:
vistext_id_to_matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4_table_captions = np.load("./vistext_test_id_to_L3_gpt4_table_captions.npy", allow_pickle=True).item()

OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
model = "TURBO"
n_errors = 0
n_passed = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    if chart_id not in vistext_id_to_gpt4_table_captions:
        print(chart_id)
        try:
            mpl_code = vistext_id_to_mpl_code[str(chart_id)]
            chart_type = vistext_id_to_chart_type[chart_id]
            exec(mpl_code)
            # gpt4-turbo
            gpt4_table_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
                                        desc_level=3, stats=["min", "max"], max_color_desc_count=0,
                                        use_starter_alt_in_prompt=False, use_azure=True,
                                        include_colors=False, include_table=True)
            plt.clf()
            vistext_id_to_gpt4_table_captions[chart_id] = gpt4_table_caption
            np.save("./vistext_test_id_to_L3_gpt4_table_captions", vistext_id_to_gpt4_table_captions)
            n_passed += 1
        except Exception as e:
            #raise e
            print(chart_id)#, mpl_code)
            n_errors += 1
            plt.clf()
            print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")

  0%|          | 0/882 [00:00<?, ?it/s]

1249


 13%|█▎        | 116/882 [00:00<00:05, 129.33it/s]

1249
Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'filtered': True, 'detected': True}}}, 'code': 'content_filter', 'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: \r\nhttps://go.microsoft.com/fwlink/?linkid=2198766.", 'param': 'prompt', 'type': None}}
1439


 15%|█▍        | 131/882 [00:08<01:00, 12.37it/s] 

...

 85%|████████▌ | 754/882 [00:50<00:08, 15.77it/s]

7361


100%|██████████| 882/882 [00:57<00:00, 15.38it/s]

Num passed: 8
Num errors: 1


<Figure size 640x480 with 0 Axes>

In [3]:
vistext_id_to_matplotalt_captions = np.load("./vistext_test_id_to_matplotalt_captions.npy", allow_pickle=True).item()
vistext_id_to_gpt4_table_alt_captions = {} #np.load("./vistext_test_id_to_L3_gpt4_table_alt_captions.npy", allow_pickle=True).item()

OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
model = "TURBO"
n_errors = 0
n_passed = 0
for chart_id in tqdm(vistext_id_to_matplotalt_captions.keys()):
    if chart_id not in vistext_id_to_gpt4_table_alt_captions:
        #print(chart_id)
        try:
            mpl_code = vistext_id_to_mpl_code[str(chart_id)]
            chart_type = vistext_id_to_chart_type[chart_id]
            exec(mpl_code)
            # gpt4-turbo
            gpt4_table_alt_caption = show_with_api_alt(OPENAI_API_KEY, model=model, methods=[], return_alt=True,
                                        desc_level=3, stats=["min", "max"], max_color_desc_count=0,
                                        use_starter_alt_in_prompt=True, use_azure=True,
                                        include_table_in_prompt=True,
                                        include_colors=False)
            plt.clf()
            vistext_id_to_gpt4_table_alt_captions[chart_id] = gpt4_table_alt_caption
            np.save("./vistext_test_id_to_L3_gpt4_table_alt_captions", vistext_id_to_gpt4_table_alt_captions)
            n_passed += 1
            #print(gpt4_table_alt_caption)
        except Exception as e:
            #raise e
            print(chart_id)#, mpl_code)
            n_errors += 1
            plt.clf()
            print(e)

print(f"Num passed: {n_passed}")
print(f"Num errors: {n_errors}")

  3%|▎         | 29/882 [03:41<1:21:47,  5.75s/it]

359
Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'filtered': True, 'detected': True}}}, 'code': 'content_filter', 'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: \r\nhttps://go.microsoft.com/fwlink/?linkid=2198766.", 'param': 'prompt', 'type': None}}


  5%|▌         | 47/882 [05:40<1:05:25,  4.70s/it]

575
Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'filtered': True, 'detected': True}}}, 'code': 'content_filter', 'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: \r\nhttps://go.microsoft.com/fwlink/?linkid=2198766.", 'param': 'prompt', 'type': None}}


 18%|█▊        | 156/882 [20:49<1:04:29,  5.33s/it]

...

 83%|████████▎ | 731/882 [1:34:20<12:11,  4.84s/it]

7117
Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'jailbreak': {'filtered': True, 'detected': True}}}, 'code': 'content_filter', 'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: \r\nhttps://go.microsoft.com/fwlink/?linkid=2198766.", 'param': 'prompt', 'type': None}}


100%|██████████| 882/882 [1:52:11<00:00,  7.63s/it]

Num passed: 867
Num errors: 15


<Figure size 640x480 with 0 Axes>